In [1]:
from collections import defaultdict
from datetime import datetime
import torch
import pandas as pd

In [2]:
ml_1m_path = '../../../../datasets/ml-1m'

In [3]:
file_path = ml_1m_path + '/movies.dat'
# file_path = Path(file_path)
data_movies = pd.read_csv(file_path, encoding='latin-1', sep='::', engine='python',
                   names=['MovieID', 'Title', 'Genres'], index_col='MovieID')
genres = data_movies['Genres'].str.get_dummies('|')
data_movies= pd.concat([data_movies, genres], axis=1)
# [row, columns], axis = 1 is column
data_movies.drop('Genres', axis=1, inplace=True)
data_movies

,Title,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
MovieID,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Jumanji (1995),0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,Grumpier Old Men (1995),0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
4,Waiting to Exhale (1995),0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
5,Father of the Bride Part II (1995),0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948,Meet the Parents (2000),0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3949,Requiem for a Dream (2000),0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3950,Tigerland (2000),0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [4]:
file_path = ml_1m_path + '/users.dat'
data_users = pd.read_csv(file_path, sep='::', engine='python',
                   names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'], index_col='UserID')

# Convert Gender to binary indicator variable
data_users['Gender'] = (data_users['Gender'] == 'M').astype(int)
occupation_map = {
    0: 'other',
    1: 'academic/educator',
    2: 'artist',
    3: 'clerical/admin',
    4: 'college/grad student',
    5: 'customer service',
    6: 'doctor/health care',
    7: 'executive/managerial',
    8: 'farmer',
    9: 'homemaker',
    10: 'K-12 student',
    11: 'lawyer',
    12: 'programmer',
    13: 'retired',
    14: 'sales/marketing',
    15: 'scientist',
    16: 'self-employed',
    17: 'technician/engineer',
    18: 'tradesman/craftsman',
    19: 'unemployed',
    20: 'writer'
}
# data['Occupation'] = data['Occupation'].map(occupation_map)
data_users

,Gender,Age,Occupation,Zip-code
UserID,,,,
1,0,1,10,48067
2,1,56,16,70072
3,1,25,15,55117
4,1,45,7,02460
5,1,25,20,55455
...,...,...,...,...
6036,0,25,15,32603
6037,0,45,1,76006
6038,0,56,1,14706


In [5]:
file_path = ml_1m_path + '/ratings.dat'

data_ratings = pd.read_csv(file_path, sep='::', engine='python',
                   names=['UserID', 'MovieID', 'Rating', 'Timestamp'], index_col='UserID')


In [6]:
data_ratings

,MovieID,Rating,Timestamp
UserID,,,
1,1193,5,978300760
1,661,3,978302109
1,914,3,978301968
1,3408,4,978300275
1,2355,5,978824291
...,...,...,...
6040,1091,1,956716541
6040,1094,5,956704887
6040,562,5,956704746


## SASRec handling
Only use implicit feedback in the sequence of items

In [10]:
data_ratings

,MovieID,Rating,Timestamp
UserID,,,
1,1193,5,978300760
1,661,3,978302109
1,914,3,978301968
1,3408,4,978300275
1,2355,5,978824291
...,...,...,...
6040,1091,1,956716541
6040,1094,5,956704887
6040,562,5,956704746


In [7]:
# use timestamps to determine the sequence order of actions.
actions = data_ratings
actions = actions.groupby('MovieID').filter(lambda  x: len(x) >= 5)
actions = actions.groupby('UserID').filter(lambda  x: len(x) >= 5)
actions = actions.groupby('UserID', group_keys=False).apply(lambda  x: x.sort_values('Timestamp'))
# We discard users and items with fewer than 5 related actions.
actions

,MovieID,Rating,Timestamp
UserID,,,
1,3186,4,978300019
1,1721,4,978300055
1,1022,5,978300055
1,1270,5,978300055
1,2340,3,978300103
...,...,...,...
6040,2917,4,997454429
6040,1784,3,997454464
6040,1921,4,997454464


In [8]:
usermap = dict()
usernum = 0
itemmap = dict()
itemnum = 0
# reorder the userid and itemid (keep the same step with original SASRec code)
for _id, row in actions.iterrows():
    if _id in usermap:
        userid = usermap[_id]
    else:
        usernum += 1
        userid = usernum
        usermap[_id] = userid

    if row.MovieID in itemmap:
        itemid = itemmap[row.MovieID]
    else:
        itemnum += 1
        itemid = itemnum
        itemmap[row.MovieID] = itemid

In [9]:
usermap.__len__(), itemmap.__len__()

(6040, 3416)

In [10]:
actions['MovieID'] = actions['MovieID'].map(itemmap)
actions.index = actions.index.map(usermap)
# data['Occupation'] = data['Occupation'].map(occupation_map)

In [15]:
actions

,MovieID,Rating,Timestamp
UserID,,,
1,1,4,978300019
1,2,4,978300055
1,3,5,978300055
1,4,5,978300055
1,5,3,978300103
...,...,...,...
6040,1249,4,997454429
6040,88,3,997454464
6040,371,4,997454464


In [16]:
sas_data = actions.drop('Rating', axis=1, inplace=False).drop('Timestamp', axis=1, inplace=False)
## ..continue
sas_data.loc[1]

,MovieID
UserID,
1,1
1,2
1,3
1,4
1,5
1,6
1,7
1,8
1,9


In [18]:
with open('ml-1m.txt', 'w') as f:
    for _id in set(sas_data.index):
        for movie_id in sas_data.loc[_id].MovieID:
            f.write('%d %d\n' % (_id, movie_id))

In [27]:
def data_partition(fname):
    usernum = 0
    itemnum = 0
    User = defaultdict(list)
    user_train = {}
    user_valid = {}
    user_test = {}
    # assume user/item index starting from 1
    f = open('%s.txt' % fname, 'r')
    for line in f:
        u, i = line.rstrip().split(' ')
        u = int(u)
        i = int(i)
        usernum = max(u, usernum)
        itemnum = max(i, itemnum)
        User[u].append(i)

    for user in User:
        nfeedback = len(User[user])
        if nfeedback < 3:
            user_train[user] = User[user]
            user_valid[user] = []
            user_test[user] = []
        else:
            user_train[user] = User[user][:-2]
            user_valid[user] = []
            user_valid[user].append(User[user][-2])
            user_test[user] = []
            user_test[user].append(User[user][-1])
    return [user_train, user_valid, user_test, usernum, itemnum]

In [49]:
dataset = data_partition('ml-1m')

In [50]:
[user_train, user_valid, user_test, usernum, itemnum] = dataset

In [24]:
len(user_train)

6040

In [25]:
len(user_valid)

6040

In [26]:
len(user_test)

6040

In [29]:
usernum

6040

In [30]:
itemnum

3416

In [36]:
import random
random.sample(range(1, usernum + 1), 10000)

[2410,
 677,
 1267,
 6109,
 512,
 1908,
 7074,
 3524,
 3296,
 1083,
 4743,
 3682,
 6252,
 874,
 1666,
 244,
 1402,
 5036,
 8103,
 1284,
 9094,
 2272,
 7757,
 1928,
 3426,
 6262,
 1180,
 1471,
 3643,
 129,
 6673,
 567,
 413,
 6180,
 9997,
 3920,
 3245,
 5370,
 9434,
 5536,
 4006,
 6363,
 466,
 280,
 1716,
 6539,
 1873,
 7427,
 5284,
 7087,
 4510,
 8527,
 3763,
 8722,
 9700,
 3981,
 5832,
 4390,
 9840,
 4337,
 7151,
 2346,
 9141,
 5940,
 548,
 2406,
 7099,
 7116,
 2647,
 80,
 323,
 7701,
 1529,
 6583,
 8432,
 9782,
 3092,
 2304,
 7077,
 6372,
 4915,
 4709,
 9496,
 3089,
 208,
 6622,
 9246,
 2357,
 3408,
 3898,
 3807,
 7442,
 7669,
 1225,
 2528,
 7759,
 9067,
 5180,
 1381,
 3019,
 3107,
 985,
 7897,
 9180,
 5139,
 2474,
 1661,
 9254,
 380,
 6085,
 3352,
 5058,
 3552,
 2627,
 1489,
 2809,
 7048,
 2891,
 868,
 4050,
 6885,
 6457,
 6058,
 7453,
 9124,
 2871,
 8120,
 1134,
 5867,
 4494,
 2846,
 3181,
 6772,
 8111,
 315,
 4396,
 737,
 6484,
 3952,
 314,
 9428,
 3974,
 1398,
 2239,
 4103,
 6294

In [55]:
import numpy as np
import copy
import sys

def evaluate_window_valid(model, dataset, args):
    [train, valid, test, usernum, itemnum] = copy.deepcopy(dataset)
    Recall = 0.0
    P90 = 0.0
    # P90 coverage means the smallest item sets that appear in the top 10 lists of at least 90% of the users.
    valid_user = 0.0
    if usernum > 10000:
        # avoid too many training users
        # keep at most 10000 users
        users = random.sample(range(1, usernum + 1), 10000)
    else:
        # else keep all the users
        users = range(1, usernum + 1)
    for u in users:
        # make sure the sequence can be validated
        if len(train[u]) < 1 or len(valid[u]) < 1: continue
        seq = np.zeros([args.maxlen], dtype=np.int32)
        idx = args.maxlen - 1
        for i in reversed(train[u]):
            seq[idx] = i
            # fill the sequence from end to beginning
            idx -= 1
            if idx == -1: break
            # select the max len or all of the training data in the sequence
            # limit the length, seq contains the actual training sequence
        rated = set(train[u])
        rated.add(0)
        # all items interacted by the current user
        item_idx = [valid[u][0]]
        # get the index of validated item
        for _ in range(100):
            # negative sampling
            t = np.random.randint(1, itemnum + 1)
            # randomly sample 100 items
            while t in rated: t = np.random.randint(1, itemnum + 1)
            item_idx.append(t)
        predictions = -model.predict(*[np.array(l) for l in [[u], [seq], item_idx]])
        # predicting the recommendation list
        predictions = predictions[0]
        rank = predictions.argsort().argsort()[0].item()
        # the rank of the expected next single item
        valid_user += 1
        if rank < 10:
            Recall += 1
            # P90 coverage
        if valid_user % 100 == 0:
            print('.', end="")
            sys.stdout.flush()
    return Recall / valid_user, P90 / valid_user

[1] [[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19
  20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43
  44 45 46 47 48 49 50 51]] [  52 3156  331 2014 1803 3311 2133 3095  270 3336 2489  911 2651 2772
 1751 1917 3184 2776  546 2559 2438 1752 2832  305 1310  358  407  832
 1822 3073 2296 2952 2984  568  545 1530 3308  288  707 1595 3303 3129
 1114 2594 1580 2964 2359 3100 1828 2273 1759 2005 1760 3018 3170 2834
 3307 2039 1405 1922 3329  563  264 1825  577 2515  586  989 1790 2676
 3096 1914 1971  851 

In [ ]:
    # rated = set(train[u])
    # rated.add(0)
    # item_idx = [valid[u][0]]
    # for _ in range(100):
    #     t = np.random.randint(1, itemnum + 1)
    #     while t in rated: t = np.random.randint(1, itemnum + 1)
    #     item_idx.append(t)
    #
    # predictions = -model.predict(*[np.array(l) for l in [[u], [seq], item_idx]])
    # predictions = predictions[0]
    #
    # rank = predictions.argsort().argsort()[0].item()
    #
    # valid_user += 1

    # if rank < 10:
    #     NDCG += 1 / np.log2(rank + 2)
    #     HT += 1
    # if valid_user % 100 == 0:
    #     print('.', end="")
    #     sys.stdout.flush()
# return NDCG / valid_user, HT / valid_user

In [72]:
item_list = []

In [73]:
item_set = set(range(1,itemnum+1))
item_set = item_set - rated
item_list+= list(item_set)


In [74]:
len(item_list)

3077